# Laboratorio 1
Edwin Ortega - 22305
Esteban Zambrano - 22119
Juan Pablo Solis - 22102

In [5]:
using JuMP
using HiGHS
using Ipopt
using Optimization

### Ejercicio 1

In [3]:
# Crear el modelo con el solver HiGHS
model = Model(HiGHS.Optimizer)

# Definir variables
@variable(model, x1 >= 0)
@variable(model, x2 >= 0)
@variable(model, x3 >= 0)
@variable(model, x4 >= 0)

# Función objetivo: maximizar z
@objective(model, Max, x1 + x2 + 3x3 + 2x4)

# Restricciones
@constraint(model, x1 + 2x2 - 3x3 + 5x4 <= 4)
@constraint(model, 5x1 - 2x2 + 6x4 <= 8)
@constraint(model, 2x1 + 3x2 - 2x3 + 3x4 <= 3)
@constraint(model, -x1 + x3 + 2x4 <= 0)

# Ejecutar el optimizador
optimize!(model)

# Mostrar resultados
println("Estado de solución: ", termination_status(model))
println("Valor óptimo de z: ", objective_value(model))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))
println("x4 = ", value(x4))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 4 cols; 14 nonzeros
Coefficient ranges:
  Matrix [1e+00, 6e+00]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [3e+00, 8e+00]
Presolving model
4 rows, 3 cols, 10 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-4); elements 0(-14) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  9.0000000000e+00
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Estado de solución: OPTIMAL
Valor óptimo de z: 9.0
x1 = 2.0
x2 = 1.0
x3 = 2.0
x4 = 0.0


### Ejercicio 3 Modelo de asignación de horarios

#### a)

In [ ]:
# -----------------------------------------------------
# a) Formulación del PL:
# Variables:
#   x_i >= 0  → autobuses que inician en el turno i
#
# Parámetros:
#   demanda = [4,8,10,7,12,4]
#   cobertura = Dict(
#     1=>[1,2], 2=>[2,3], 3=>[3,4],
#     4=>[4,5], 5=>[5,6], 6=>[6,1]
#   )
#
# Objetivo:
#   min Σ x_i
#
# Restricciones:
#   x1 + x6 >= 4   # intervalo 1
#   x1 + x2 >= 8   # intervalo 2
#   x2 + x3 >=10   # intervalo 3
#   x3 + x4 >= 7   # intervalo 4
#   x4 + x5 >=12   # intervalo 5
#   x5 + x6 >= 4   # intervalo 6
# -----------------------------------------------------

#### b)

In [8]:
# — Datos —
turnos     = 1:6
intervalos = 1:6

cobertura = Dict(
    1 => [1, 2],  # 00:00–07:59
    2 => [2, 3],  # 04:00–11:59
    3 => [3, 4],  # 08:00–15:59
    4 => [4, 5],  # 12:00–19:59
    5 => [5, 6],  # 16:00–23:59
    6 => [6, 1]   # 20:00–03:59
)

demanda = [4, 8, 10, 7, 12, 4]  # según Figura 2

# — Crear modelo con HI‑GHS como solver —
model = Model(HiGHS.Optimizer)

# Variables: x[i] = autobuses que inician en turno i
@variable(model, x[turnos] >= 0)

# Restricciones: cubrir la demanda en cada intervalo
@constraint(model, [j in intervalos],
    sum(x[i] for i in turnos if j in cobertura[i]) >= demanda[j]
)

# Objetivo: minimizar total de autobuses
@objective(model, Min, sum(x[i] for i in turnos))

# Resolver
optimize!(model)

# Resultados
println("Estado de solución: ", termination_status(model))
println("Total de autobuses necesarios: ", objective_value(model))
for i in turnos
    println("  Turno $i → ", value(x[i]) |> round, " autobuses")
end

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 6 cols; 12 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 1e+01]
Presolving model
6 rows, 6 cols, 12 nonzeros  0s
6 rows, 6 cols, 12 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 6(-0); elements 12(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(45) 0s
          5     2.6000000000e+01 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 5
Objective value     :  2.6000000000e+01
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Estado de solución: OPTIMAL
Total de autobuses necesarios: 26.0
  Turno 1 → 4.0 autobuses
  Turno 2 → 4.0 autobuses
  Turno 3 → 6.0 autobuses
  Turno 4 → 1.0 autobuses
  Turno 5 → 11.0 a